In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import pandas as pd
import sys
import datetime
import pandas as pd

from textblob.classifiers import NaiveBayesClassifier as NBC
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

class TwitterClient(object):
	'''
	Generic Twitter Class for sentiment analysis.
	'''
	def __init__(self):
		'''
		Class constructor or initialization method.
		'''
		# keys and tokens from the Twitter Dev Console
		consumer_key = 'Input key'
		consumer_secret = 'Input key'
		access_token = 'Input key'
		access_token_secret = 'Input key'

		# attempt authentication
		try:
			# create OAuthHandler object
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			# set access token and secret
			self.auth.set_access_token(access_token, access_token_secret)
			# create tweepy API object to fetch tweets
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		'''
		Utility function to clean tweet text by removing links, special characters
		using simple regex statements.
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

	def get_tweet_sentiment(self, tweet):
		'''
		Utility function to classify sentiment of passed tweet
		using textblob's sentiment method
		'''
		# create TextBlob object of passed tweet text
		analysis = TextBlob(self.clean_tweet(tweet))
		# set sentiment
		if analysis.sentiment.polarity > 0:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets(self, query,lang,count):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []

		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search(q= query, lang = lang, count=count)

			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}

				parsed_tweet['created_at'] = tweet.created_at
				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)

			# return parsed tweets
			return tweets

		except tweepy.TweepError as e:
			# print error (if any)
			print("Error : " + str(e))

	def get_tweets_time(self, env, query, since, until):
		'''
		Main function to fetch tweets and parse them.
		'''
		# empty list to store parsed tweets
		tweets = []

		try:
			# call twitter api to fetch tweets
			fetched_tweets = self.api.search_full_archive(environment_name = env, query = query,fromDate= since, toDate= until)

			# parsing tweets one by one
			for tweet in fetched_tweets:
				# empty dictionary to store required params of a tweet
				parsed_tweet = {}

				parsed_tweet['created_at'] = tweet.created_at
				# saving text of tweet
				parsed_tweet['text'] = tweet.text
				# saving sentiment of tweet
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

				# appending parsed tweet to tweets list
				if tweet.retweet_count > 0:
					# if tweet has retweets, ensure that it is appended only once
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)

			# return parsed tweets
			return tweets

		except tweepy.TweepError as e:
			# print error (if any)
			print("Error : " + str(e))

def main():
	# creating object of TwitterClient Class
	api = TwitterClient()
	keyword = 'GE14' #'malaysiamemilih' and 'GE14' and 'pru14' and 'battle4my' and 'myundi'#input("Please enter keyword or hashtag to search: ")
	date_since='201703102359' #kalau nak guna fix time tukar dr sini ja. kalau mau input guna input function copy jak dr comment
	date_until='201705080000' #input("Please enter end time: ")
	language = 'en' #input("Please enter language prefer: ") en = english my/msa = malay
	# calling function to get tweets latest 1 week 
	tweets = api.get_tweets(query = keyword, lang=language,count=100) #normal
	#calling with full archive 
	tweets = api.get_tweets_time(env= 'Hello', query = (keyword),since = date_since,until = date_until) #set the time 

	#print(tweets)
	creat = []
	texts = []
	sen = []
	for tweet in tweets:
		#print(tweet['created_at'],tweet['text'],tweet['sentiment']) 
		creat.append(tweet['created_at'])
		texts.append(tweet['text'])
		sen.append(tweet['sentiment'])
	data={'Create_at': creat, 'Text':texts,'Sentiment':sen}
	result = pd.DataFrame(data, columns= ['Create_at','Text','Sentiment'])
	#print(result)
	writer = pd.ExcelWriter('./original.xlsx')
	result.to_excel(writer)
	writer.save()
	#print("success save data in excel")
  
  #call the excel
	df = pd.read_excel('original.xlsx')
	print(len(df)) 
	import re
  
	def deEmojify(text):
		regrex_pattern = re.compile(pattern = "["
				u"\U0001F600-\U0001F64F"  # emoticons
				u"\U0001F300-\U0001F5FF"  # symbols & pictographs
				u"\U0001F680-\U0001F6FF"  # transport & map symbols
				u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
														"]+", flags = re.UNICODE)
		return regrex_pattern.sub(r'',text)

	for i in range(len(df)):
		txt = df.loc[i]["Text"]
		txt = deEmojify(txt)
		txt=re.sub(r'@[A-Z0-9a-z_:]+','',txt)#replace username-tags
		txt=re.sub(r'^[RT ]+','',txt)#replace RT-tags
		txt = re.sub('https?://[A-Za-z0-9./]+','',txt)#replace URLs
		txt = re.sub('\'','',txt) #remove symbol
		txt = re.sub('\’','',txt) #remove symbol
		txt = re.sub('\‘',' ',txt) #remove symbol
		txt = re.sub('\…','',txt) #remove symbol
		txt = re.sub('\🤣','',txt) #remove symbol
		txt = re.sub('\n','',txt) #remove enter
		txt = txt.rstrip() #remove space behind sentences
		#txt = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", txt) #remove number
		#txt=re.sub("#", "",txt)#replace hashtags
		df.at[i,"Text"]=txt

	nan_value = float('NaN')
	df.replace('', nan_value, inplace=True)
	df.dropna(subset = ['Text'], inplace=True)
	df = df.drop_duplicates(subset=['Text'])

	#save remove duplicate and retweet
	result2 = pd.DataFrame(df, columns= ['Create_at','Text','Sentiment'])
	writer2 = pd.ExcelWriter('./pre-processing.xlsx') #after preprocessing
	result2.to_excel(writer2)
	writer2.save()  
	print("success save data in excel") 
	#call the excel
	df2 = pd.read_excel('./pre-processing.xlsx')
	result2 = pd.DataFrame(df2, columns= ['Create_at','Text','Sentiment'])
	writer2 = pd.ExcelWriter('./removeduplicate.xlsx')
	result2.to_excel(writer2)
	writer2.save()
	print(len(df2))
 
	print("\n ")
	print("THE NUMBER OF TWEET EXTRACT FROM HASHTAG = " + str(len(df2))) #how many number can extract based on the tweeter package u used sandbox only 100 , search only 1 week latest data
	if len(df2)==0:
		print('There is no data for this language')
	else:
		Total = len(df2.Sentiment)
		ptweet = 0
		ntweet = 0
		neutral = 0
		for i in range(len(df2)):
			txt = df2.loc[i]["Sentiment"]
			if txt == 'positive':
				ptweet += 1
			elif txt == 'negative':
				ntweet += 1
			else: 
				neutral +=1

		print("Positive tweets percentage: " + str(100*(ptweet/Total)) + '%')
		print("Negative tweets percentage: " + str(100*(ntweet/Total)) + '%')
		print("Neutral tweets percentage: " + str(100*(neutral/Total)) + '%')

if __name__ == "__main__":
	# calling main function
	main()


100
success save data in excel
60

 
THE NUMBER OF TWEET EXTRACT FROM HASHTAG = 60
Positive tweets percentage: 35.0%
Negative tweets percentage: 11.666666666666666%
Neutral tweets percentage: 53.333333333333336%
